In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import numpy as np
import geopandas as gpd
from tqdm.notebook import tqdm
from scripts import contour_gen

In [ ]:
endpoint = 'http://api.plastic.watch.earthrise.media/sites?limit=10000'
response = requests.get(endpoint)
data = response.json()
data = gpd.GeoDataFrame.from_features(data['features'])
for c in data.columns:
    try:
        data[c] = data[c].astype(float)
    except:
        pass

In [ ]:
threshold = 18
auth = requests.auth.HTTPBasicAuth('admin', 'plastics')
for site_id in tqdm(data['id']):
    endpoint = f'http://api.plastic.watch.earthrise.media/sites/{site_id}/contours'
    response = requests.get(endpoint)
    response = response.json()
    gdf = gpd.GeoDataFrame.from_features(response['features'])
    if len(gdf) > 1:
        areas = gdf['area (km^2)'].astype(float)
        std, mean = np.std(areas) / np.sqrt(len(areas)), np.mean(areas)
        # calculate number of stdevs from mean for each value in areas
        stdevs = [(area - mean) / std for area in areas]
        if np.max(stdevs) > threshold:
            print("Outliers detected at site", site_id)
        while np.max(stdevs) > threshold:
            response = requests.get(endpoint)
            response = response.json()
            gdf = gpd.GeoDataFrame.from_features(response['features'])
            areas = gdf['area (km^2)'].astype(float)
            std, mean = np.std(areas) / np.sqrt(len(areas)), np.mean(areas)
            stdevs = [(area - mean) / std for area in areas]
            outlier_indices = np.where(np.abs(stdevs) > threshold)[0]
            outlier_ids = [response['features'][index]['id'] for index in outlier_indices]
            for contour_id in outlier_ids:
                contour_endpoint = f'http://api.plastic.watch.earthrise.media/sites/{site_id}/contours/{contour_id}'
                response = requests.delete(contour_endpoint, auth=auth)
                #print(site_id, contour_id, response.status_code)